In [19]:
!pip install -U google-generativeai weaviate-client faiss-cpu numpy rank-bm25 nltk --quiet

/home/mrittika/.pyenv/versions/3.12.10/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=13409) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.31 requires torch==2.7.1, but you have torch 2.6.0+cpu which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
numba 0.61.2 requires numpy<2.3,>=1.24, but you have numpy 2.3.3 which is incompatible.


In [20]:
import os
import pickle
import numpy as np
import google.generativeai as genai
import weaviate
from weaviate.classes.config import Configure
import nltk
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

In [ ]:
# ===============================
API_KEY = "GOOGLE_API"  # <-- replace with your key
genai.configure(api_key=API_KEY)


In [ ]:
# ===============================
WCS_URL = "WEAVIATE_URL"  # <-- replace
WCS_API_KEY = "WEAVIATE_API_KEY"

In [24]:
client = weaviate.connect_to_wcs(
    cluster_url=WCS_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCS_API_KEY),
)

class_name = "PhysicsChunk"

# Optional: reset collection
if client.collections.exists(class_name):
    client.collections.delete(class_name)

client.collections.create(
    class_name,
    vectorizer_config=Configure.Vectorizer.none(),  # we provide vectors
    vector_index_config=Configure.VectorIndex.hnsw()
)

collection = client.collections.get(class_name)


/tmp/ipykernel_13409/3428761974.py:1: DeprecatedWarning: connect_to_wcs is deprecated as of 4.6.2. 
This method is deprecated and will be removed in a future release. Use :func:`connect_to_weaviate_cloud` instead.

  client = weaviate.connect_to_wcs(
/home/mrittika/.pyenv/versions/test/lib/python3.12/site-packages/deprecation.py:260: DeprecationWarning: This method is deprecated and will be removed in a future release. Use `connect_to_weaviate_cloud` instead.
  return function(*args, **kwargs)
/home/mrittika/.pyenv/versions/test/lib/python3.12/site-packages/weaviate/warnings.py:196: DeprecationWarning: Dep024: You are using the `vectorizer_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead.
            
  warnings.warn(
/home/mrittika/.pyenv/versions/test/lib/python3.12/site-packages/weaviate/warnings.py:206: DeprecationWarning: Dep025: You are using the `vector_index_config` argument in `collection.config.create(

In [25]:
# Step 2. Load file and split into chunks
# ===============================
file_path = '/home/mrittika/projects/eduRAG/physics_embeddings/Physics/combined_physics.md'


In [26]:
file_path = '/home/mrittika/projects/eduRAG/physics_embeddings/Physics/combined_physics.md'

with open(file_path, 'r', encoding='utf-8') as f:
    content = f.read()

chunks = [t.strip() for t in content.split('*****') if t.strip()]
print(f"Total number of chunks: {len(chunks)}")

Total number of chunks: 407


In [27]:
# Step 4. Generate Embeddings
# ===============================
result = genai.embed_content(
    model="models/gemini-embedding-001",
    content=chunks
)
embeddings = np.array(result['embedding'], dtype='float32')
print("Embeddings generated.")


E0000 00:00:1758090263.352122   13409 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Embeddings generated.


In [28]:
for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
    collection.data.insert(
        properties={"text": chunk},
        vector=emb.tolist()
    )

print(f"Inserted {len(chunks)} chunks into Weaviate Cloud.")


Inserted 407 chunks into Weaviate Cloud.


In [30]:
# ===============================
query_text = "পদার্থবিজ্ঞানের শাখা গুলো কি কি ?"

# Get embedding for query from Gemini
query_embedding = genai.embed_content(
    model="models/gemini-embedding-001",
    content=query_text
)["embedding"]

# Pass BOTH text + vector to hybrid search
results = collection.query.hybrid(
    query=query_text,
    vector=query_embedding,  # <--- this fixes the error
    alpha=0.5,               # balance keyword vs vector
    limit=5
)

print("\n--- Hybrid Search Results (Weaviate Cloud) ---")
top_docs = []
for rank, obj in enumerate(results.objects):
    text_chunk = obj.properties['text']
    top_docs.append(text_chunk)
    print(f"Rank {rank+1}: {text_chunk[:200]}...\n")



--- Hybrid Search Results (Weaviate Cloud) ---
Rank 1: ## অধ্যায় 1: ভৌত রাশি এবং তাদের পরিমাপ

## 1.2 পদার্থবিজ্ঞানের পরিসর (Scope of Physics)

পদার্থবিজ্ঞান যেহেতু বিজ্ঞানের প্রাচীনতম শাখা এবং সবচেয়ে মৌলিক শাখা, শুধু তা-ই নয়, বিজ্ঞানের অন্যান্য শাখা ক...

Rank 2: ## অধ্যায় 1: ভৌত রাশি এবং তাদের পরিমাপ

## 1.1 পদার্থবিজ্ঞান (Physics)

পদার্থবিজ্ঞান বিজ্ঞানের একটা শাখা এবং বলা যেতে পারে এটা হচ্ছে প্রাচীনতম শাখা। তার কারণ অন্য বিজ্ঞানগুলো দানা বাঁধার অনেক আগেই ব...

Rank 3: ## অধ্যায় 1: ভৌত রাশি এবং তাদের পরিমাপ

## 1.4 পদার্থবিজ্ঞানের উদ্দেশ্য (Objectives of Physics)

তোমরা এর মাঝে জেনে গেছ যে পদার্থবিজ্ঞান হচ্ছে বিজ্ঞানের সেই শাখা, যেটি শক্তি এবং বলের উপস্থিতিতে সময়ে...

Rank 4: ## অধ্যায় 1: ভৌত রাশি এবং তাদের পরিমাপ

## এ অধ্যায় শেষে আমরা-

- পদার্থবিজ্ঞানের পরিসর ও ক্রমবিকাশ ব্যাখ্যা করতে পারব।
- পদার্থবিজ্ঞান পাঠের উদ্দেশ্য বর্ণনা করতে পারব।
- ভৌত রাশি [মান এবং এককসহ] ও ...

Rank 5: ## অধ্যায় 1: ভৌত রাশি এবং তাদের পরিমাপ

## (Physical Quantities And Their Measurement)

![]

In [31]:
# ===============================
# Step 7. Generate Answer with Gemini
# ===============================
top_document = top_docs[0] if top_docs else ""

response_prompt = f"""
তুমি একজন বাংলা পদার্থবিজ্ঞানের শিক্ষক। এই প্রসঙ্গ ব্যবহার করে প্রশ্নটির উত্তর দাও:

প্রশ্ন: {query_text}

প্রসঙ্গ: {top_document}

উত্তর বাংলায় দাও এবং শিক্ষার্থীদের জন্য সহজবোধ্য করে ব্যাখ্যা করো।
"""

try:
    response = genai.GenerativeModel('gemini-2.5-flash').generate_content(response_prompt)
    print("=== Generated Response ===")
    print(response.text)
except Exception as e:
    print(f"Error generating response: {e}")

=== Generated Response ===
প্রিয় শিক্ষার্থীরা, আজকের এই প্রশ্নটি খুবই মৌলিক এবং অত্যন্ত গুরুত্বপূর্ণ। পদার্থবিজ্ঞান যেহেতু আমাদের চারপাশের জগৎ এবং মহাবিশ্বকে বোঝার জন্য বিজ্ঞানের সবচেয়ে মৌলিক শাখা, তাই এর পরিসরও সুবিশাল। পঠন-পাঠনের সুবিধার জন্য এবং এর বিশাল জগতকে সহজে বোঝার জন্য, আমরা পদার্থবিজ্ঞানের শাখাগুলোকে প্রধানত দুটি মূল ভাগে ভাগ করে থাকি। এসো, আমরা সেগুলো নিয়ে বিস্তারিত আলোচনা করি:

### ১. চিরায়ত পদার্থবিজ্ঞান (Classical Physics)

এটি পদার্থবিজ্ঞানের সেই শাখা যা বিংশ শতাব্দীর শুরুর দিকের আগে পর্যন্ত গঠিত হয়েছিল। এটি মূলত আমাদের দৈনন্দিন জীবনের বড় বড় ঘটনাগুলো ব্যাখ্যা করে এবং ম্যাক্রোস্কোপিক (চোখে দেখা যায় এমন) বস্তুর আচরণ নিয়ে কাজ করে। এর অন্তর্ভুক্ত প্রধান শাখাগুলো হলো:

*   **বলবিজ্ঞান (Mechanics):** এটি বস্তুর গতি, শক্তি এবং বল নিয়ে আলোচনা করে। একটি পাথর উপরে ছুঁড়লে সেটি কীভাবে মাটিতে পড়ে, গ্রহ-নক্ষত্র কীভাবে ঘোরে, বা একটি গাড়ি কীভাবে চলে – এ সবই বলবিজ্ঞানের আলোচনার বিষয়।
*   **শব্দবিজ্ঞান (Acoustics):** শব্দের উৎপত্তি, এর সঞ্চালন এবং এর বিভিন্ন ধর্ম যেমন কম্পা